# Double Texting

Seamless handling of [double texting](https://langchain-ai.github.io/langgraph/concepts/double_texting/) is important for handling real-world usage scenarios, especially in chat applications.

Users can send multiple messages in a row before the prior run(s) complete, and we want to ensure that we handle this gracefully.

## Reject

A simple approach is to [reject](https://langchain-ai.github.io/langgraph/cloud/how-tos/reject_concurrent/) any new runs until the current run completes.

In [ ]:
%%capture --no-stderr
%pip install -U langgraph_sdk

In [1]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [2]:
import httpx
from langchain_core.messages import HumanMessage

# Create a thread
thread = await client.threads.create()

# Create to dos
user_input_1 = "Add a ToDo to follow-up with DI Repairs."
user_input_2 = "Add a ToDo to mount dresser to the wall."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)
try:
    await client.runs.create(
        thread["thread_id"],
        graph_name,
        input={"messages": [HumanMessage(content=user_input_2)]}, 
        config=config,
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/3e2f44a2-ba15-4017-bfc0-05df8212dbad/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


In [3]:
from langchain_core.messages import convert_to_messages

# Wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Add a ToDo to follow-up with DI Repairs.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_jUi4eEKN9Hhr1TIc1AqaSghf)
 Call ID: call_jUi4eEKN9Hhr1TIc1AqaSghf
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Follow-up with DI Repairs', 'time_to_complete': 10, 'solutions': ['Call DI Repairs to check on the status of the repair', 'Email DI Repairs for a written update', 'Review any previous communications with DI Repairs'], 'status': 'not started'}
================================== Ai Message ==================================

I've added a new task to follow-up with DI Repairs to your ToDo list. If you need any more help, just let me know!


## Enqueue

We can use [enqueue](https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/) any new runs until the current run completes.

In [4]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Send Erik his t-shirt gift this weekend."
user_input_2 = "Get cash and pay nanny for 2 weeks. Do this by Friday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

first_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="enqueue",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Add a ToDo to follow-up with DI Repairs.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_jUi4eEKN9Hhr1TIc1AqaSghf)
 Call ID: call_jUi4eEKN9Hhr1TIc1AqaSghf
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Follow-up with DI Repairs', 'time_to_complete': 10, 'solutions': ['Call DI Repairs to check on the status of the repair', 'Email DI Repairs for a written update', 'Review any previous communications with DI Repairs'], 'status': 'not started'}
================================== Ai Message ==================================

I've added a new task to follow-up with DI Repairs to your ToDo list. If you need any more help, just let me know!
================================ Human Message =================================

Send Erik his t-shirt gift

## Interrupt

We can use [interrupt](https://langchain-ai.github.io/langgraph/cloud/how-tos/interrupt_concurrent/) to interrupt the current run, but save all the work that has been done so far up to that point.


In [5]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Order turkey for Thanksgiving by Friday."
user_input_2 = "Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

interrupted_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="interrupt",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_ti2tr2wXga3pA0WokTi5ZD3z)
 Call ID: call_ti2tr2wXga3pA0WokTi5ZD3z
  Args:
    update_type: todo
================================= Tool Message =================================

Document f27e786f-c3ff-450c-b2e7-ded273a44721 updated:
Plan: Update the deadline for the task 'Call parents back about Thanksgiving plans' to reflect the correct date of Thanksgiving, which is November 28, 2024. This involves changing the 'deadline' field from '2024-11-23T23:59:59' to '2024-11-28T23:59:59'.
Added content: 2024-11-28T23:59:59

New ToDo created:
Content: {'task': 'Order Ham for Thanksgiving', 'time_to_complete': 20, 'deadline': '2024-11-22T23:59:59', 'solutions': ['Choose a ham supplier', 'Decide on the type and size of ham'

In [6]:
# Confirm that the first run was interrupted
print((await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"])

interrupted


## Rollback

We can use [rollback](https://langchain-ai.github.io/langgraph/cloud/how-tos/rollback_concurrent/) to interrupt the prior run of the graph and starts a new one with the double-texted input.


In [15]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Add a ToDo to call to make appointment at Yoga."
user_input_2 = "Actually, add a ToDo to drop by Yoga in person on Sunday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

rolled_back_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="rollback",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Actually, add a ToDo to drop by Yoga in person on Sunday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_4pJxJmwdJRWOqOzfLOjnt1vD)
 Call ID: call_4pJxJmwdJRWOqOzfLOjnt1vD
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Drop by Yoga in person on Sunday', 'time_to_complete': 30, 'deadline': '2024-11-17T23:59:59', 'solutions': ["Check the yoga studio's schedule for Sunday", 'Prepare any necessary equipment or attire', 'Plan transportation to the studio', 'Consider inviting a friend to join'], 'status': 'not started'}
================================== Ai Message ==================================

I've added the task to drop by Yoga in person on Sunday to your ToDo list. If there's anything else you need, feel free to let me know!


In [16]:
# Confirm that the original run was deleted
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as _:
    print("Original run was correctly deleted")

In [17]:
state = await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
state


{'run_id': '1efa2141-324b-6924-a03f-39aeb186e29b',
 'thread_id': '6d105043-6cbd-4d88-a26a-b648b65efd29',
 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21',
 'created_at': '2024-11-13T23:07:37.599504+00:00',
 'updated_at': '2024-11-13T23:07:37.599504+00:00',
 'metadata': {},
 'status': 'interrupted',
 'kwargs': {'input': {'messages': [{'id': None,
     'name': None,
     'type': 'human',
     'content': 'Add a ToDo to call to make appointment at Yoga.',
     'example': False,
     'additional_kwargs': {},
     'response_metadata': {}}]},
  'config': {'metadata': {'created_by': 'system'},
   'configurable': {'run_id': '1efa2141-324b-6924-a03f-39aeb186e29b',
    'user_id': 'Lance',
    'graph_id': 'task_maistro',
    'thread_id': '6d105043-6cbd-4d88-a26a-b648b65efd29',
    'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}},
  'webhook': None,
  'subgraphs': False,
  'temporary': False,
  'stream_mode': ['values'],
  'feedback_keys': None,
  'interrupt_after': None,
  'interrup